    # What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from Kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications, we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary.

Given that our stakeholders are seeking to understand what factors drive the price of a used car then their goal is to improve their sales and meet market demand. They have provided as a dataset of used car sales and have asked us to determine which factors make a car more or less expensive. A recent [report](https://www.statista.com/statistics/183713/value-of-us-passenger-cas-sales-and-leases-since-1990/) shows that used car sales doubles the amount of new car sales. The used car market is lucrative with high demand from consumers. Our stakeholder wants to stay ahead of the curve and understanding their customer's preferences can help lead to more sales. Used car dealerships also have to acquire cars for inventory from other entities. If our stakeholder can reasonably determine the cost of a used vehicle they can better understand their margins and when to purchase a vehicle for inventory. It is also important to understand what customers value in used cars. Knowing what drives a consumer to purchase a used vehicle can help our stakeholders avoid costly mistakes when acquiring inventory.

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [ ]:
import os.path

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

from plotly.subplots import make_subplots
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from warnings import filterwarnings

filterwarnings('ignore')

In [ ]:
# Load data
vehicles_df = pd.read_csv('data/vehicles.csv')

In [ ]:
# First we should get the basic facts of the data (describe, info, null checks, duplicate checks)
vehicles_df.info()

The id and VIN column are not important for analysis so we will definitely remove them.

In [ ]:
vehicles_df.describe()

In [ ]:
vehicles_df.describe(include='object')

In [ ]:
print(f'Row count: {vehicles_df.shape[0]}, Duplicate count: {vehicles_df.shape[0] - vehicles_df.drop_duplicates().shape[0]}')

In [ ]:
vehicles_df.isna().mean().round(2)

In [ ]:
vehicles_df.sample(5)

Luckily, we do not have duplicates. However, we are missing values, specifically the condition and cylinders attributes are missing in over 40% of rows. Color is missing in 31% of rows. Size is missing in 72% of rows. Drive is missing in 31% of rows. I know that these attributes are important in determining the price of a car from prior analysis. The amount of missing data is concerning and the best course of action may be to impute them. That may mislead our model and we should be wary of that. However, dropping over 40% of the data isn't something we should do. One way to impute this data would be to find a finding an identical year and model then we would select the most seen value for that respective column; we can leverage scipy's `SimpleImputer`. This can help us deduce the missing attributes.

I will create an imputed dataset and run my analysis on both of them.

The imputed vehicle dataframe is looking a lot better.
<table>
<tr>
<th>column</th>
<th>old missing value percent</th>
<th>new missing value percent</th>
</tr>
<tr>
<td>condition</td>
<td>0.41</td>
<td>0.09</td>
</tr>
<tr>
<td>condition</td>
<td>0.42</td>
<td>0.14</td>
</tr>
<td>drive</td>
<td>0.31</td>
<td>0.10</td>
</tr>
</tr>
<td>size</td>
<td>0.72</td>
<td>0.30</td>
</tr>
</tr>
<td>type</td>
<td>0.22</td>
<td>0.05</td>
</tr>
</tr>
<td>paint_color</td>
<td>0.31</td>
<td>0.08</td>
</tr>
</table?


Let's start by checking a histogram of sales prices

In [ ]:
fig = px.box(vehicles_df, x='price', title='Price attribute has unrealistic values',
                   labels={'price': 'Price', 'count': 'Count'})
fig.show()
fig.write_image('images/box_amount_with_outliers.png')

In [ ]:
fig = px.box(vehicles_df, x='year', labels={'year': 'Year', 'count': 'Count'}, title="Most cars sold are 2008 or older models")
fig.show()
fig.write_image('images/year_box_with_outliers.png')

In [ ]:
# predefine to avoid extra computation
groupby_model_year = vehicles_df.groupby(['model', 'year'])
fields_to_impute = ['condition', 'cylinders', 'drive', 'size', 'type', 'paint_color']

# This somewhat mimics SimpleImputer. If we wanted to get fancy we would use KNN and leverage sales price in imputation
def impute_field_by_model_year(row, field_to_update):
    # ensure fields are not na and some non na value exists
    try:
        # take the most common occurrences for model and year
        group = groupby_model_year.get_group((row['model'], row['year']))
        non_na_values = group[field_to_update].dropna()
        if not non_na_values.empty:
            return non_na_values.mode().iloc[0]
    except KeyError:
        return np.nan
    return np.nan

# Impute once since it can be expensive
def load_imputed_data():
    if not os.path.exists('data/imputed_vehicles.csv'):
        # copy vehicles df and impute columns missing values
        imputed_vehicle_df = vehicles_df.copy(deep=True)
        for field in fields_to_impute:
            na_filter = imputed_vehicle_df[field].isna()
            imputed_vehicle_df.loc[na_filter, field] = imputed_vehicle_df[na_filter].apply(lambda row: impute_field_by_model_year(row, field), axis=1)

        imputed_vehicle_df.to_csv('data/imputed_vehicles.csv')
    else:
        imputed_vehicle_df = pd.read_csv('data/imputed_vehicles.csv')
    return imputed_vehicle_df

imputed_vehicle_df = load_imputed_data()

In [ ]:
imputed_vehicle_df.isna().mean().round(2)

The above is not realistic. We are seeing vehicles selling for billions. I will remove outliers using iqr. We also see very old cars (likely vintage) in the outlier group for year. I will remove those as well.

In [ ]:
q1 = vehicles_df['price'].quantile(0.25)
q3 = vehicles_df['price'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr)
upper_bound = q3 + (1.5 * iqr)

vehicles_df_no_outlier = vehicles_df.query(f'price >= {lower_bound} and price <= {upper_bound} and year > 1995')
imputed_vehicle_df = imputed_vehicle_df.query(f'price >= {lower_bound} and price <= {upper_bound} and year > 1995')

In [ ]:
# Now we can drop na
vehicles_df_no_outlier.dropna(inplace=True)
imputed_vehicle_df.dropna(inplace=True)
print(vehicles_df_no_outlier.shape[0])
print(imputed_vehicle_df.shape[0])

After removing outliers & dropping rows with NAs the non imputed dataset contains 33,728 rows and the imputed set contains 148,360. This is out of the original 426,880 rows.

In [ ]:
fig = px.box(vehicles_df_no_outlier, x='price', title='Majority of vehicles fall between &#36;6,500 - &#36;21,990',
                   labels={'price': 'Price', 'count': 'Count'})
fig.show()
fig.write_image('images/box_amount.png')

In [ ]:
fig = px.histogram(vehicles_df_no_outlier, x='price', nbins=10, labels={'price': 'Price', 'count': 'Count'}, title="A majority of vehicles are bought at a lower price")
fig.show()
fig.write_image('images/price_histogram.png')

In [ ]:
fig = px.histogram(vehicles_df_no_outlier, x='year', nbins=10, labels={'year': 'Year', 'count': 'Count'}, title="Most cars sold are newer")
fig.show()
fig.write_image('images/year_histogram.png')

In [ ]:
# Univariate analysis with the imputed columns. Compare between non imputed vs imputed
fields_to_impute = ['condition', 'cylinders', 'drive', 'size', 'type', 'paint_color']

fig = make_subplots(
    rows=len(fields_to_impute), cols=2,
    subplot_titles=("Condition: Good and excellent were sold the most",
                    "Condition: Imputation increased the amount of excellent vehicles, should be cautious",
                    "Cylinders: 6, 4 & 8 cylinders sold the most",
                    "Cylinders: Imputation follows the ratio",
                    "Drive: 4 wheel drive is the most popular",
                    "Drive: Imputation follows the ratio",
                    "Color: White, black and silver make the majority of the sales",
                    "Color: Imputation follows the ratio",
                    "Size: Full size is the most popular",
                    "Size: Imputation follows the ratio",
                    "Type: Sedan, SUVs and pickups make the majority of the sales",
                    "Type: Imputation shuffled coupes, hatchbacks, convertibles and mini-vans",
                    )
)

for i, field in enumerate(sorted(fields_to_impute)):
    no_outlier_counts = vehicles_df_no_outlier[field].value_counts()
    bar_vehicle_fig = go.Bar(x=no_outlier_counts.index, y=no_outlier_counts)
    fig.add_trace(bar_vehicle_fig, row=i+1, col=1)

    imputed_counts = imputed_vehicle_df[field].value_counts()
    bar_imputed_fig = go.Bar(x=imputed_counts.index, y=imputed_counts)
    fig.add_trace(bar_imputed_fig, row=i+1, col=2)

fig.update_layout(height=1400, width=1400, showlegend=False)
fig.show()
fig.write_image('images/imputed_fields_comparison.png')

Imputation had some effect on our data. In particular, condition and type shuffled in terms of sales count. This is important because condition is so instrumental in determining sales price and the imputation may affect our model. We saw that type saw impact in the types with lower sales and that may not affect our model. We will create models for both datasets and use metrics to determine if our imputation helped or hurt our cause.

In [ ]:
#Univariate analysis with the non-imputed columns
non_altered_columns = list(filter(lambda field: field not in fields_to_impute and field != "VIN", list(vehicles_df_no_outlier.select_dtypes(include='object').columns)))
fig = make_subplots(
    rows=len(non_altered_columns), cols=1,
    subplot_titles=("Fuel: Gas is the most popular type",
                    "Manufacturer: Ford, Chevy and Toyota are the most popular manufacturer",
                    "Model: Preference is scattered. F150 tops. We see normalization of model names was not done (see silverado)",
                    "Region: Should represent this differently",
                    "State: Most populous states have most sales as expected. Should represent this differently",
                    "Title status: Clean title is preferred",
                    "Transmission: Automatic is the most popular",
                    )
)

for i, field in enumerate(sorted(non_altered_columns)):
    no_outlier_counts = vehicles_df_no_outlier[field].value_counts()
    # if a value represent less than 1% of the rows we will put it in the other column
    total_counts = no_outlier_counts.sum()
    percent_threshold = 0.01 # 1%
    other_mask = (no_outlier_counts / total_counts) < percent_threshold
    other_sum = no_outlier_counts[other_mask].sum()
    no_outlier_counts_filtered = no_outlier_counts[~other_mask]
    if 'other' in list(no_outlier_counts_filtered.index):
        no_outlier_counts_filtered['other'] = other_sum + no_outlier_counts_filtered['other']
    else:
        no_outlier_counts_filtered['other'] = other_sum
    bar_vehicle_fig = go.Bar(x=no_outlier_counts_filtered.index, y=no_outlier_counts_filtered)
    fig.add_trace(bar_vehicle_fig, row=i+1, col=1)

fig.update_layout(height=1400, width=1400, showlegend=False)
fig.show()
fig.write_image('images/univariate_non_imputed_fields_bar.png')

The above analysis shows that the region, model and state should be represented differently.


In [ ]:
fig = px.treemap(vehicles_df_no_outlier, path=[px.Constant("USA"), 'state', 'region'], values='price',
                  color='price', hover_data=['region'],
                  color_continuous_scale='Magma',
                  color_continuous_midpoint=np.average(vehicles_df_no_outlier['price']), title='California has highest volume of sales and North Carolina has highest average')
fig.show()
fig.write_image('images/treemap_no_outlier.png')

In [ ]:
fig = px.treemap(imputed_vehicle_df, path=[px.Constant("USA"), 'state', 'region'], values='price',
                  color='price', hover_data=['region'],
                  color_continuous_scale='Magma',
                  color_continuous_midpoint=np.average(imputed_vehicle_df['price']), title='California retains highest volume of sales but now Washington has highest average')
fig.show()
fig.write_image('images/treemap_imputated.png')

With the treemaps we see that the most populated states have the highest volume. However, other states have a higher price average. This could be indicative of different needs in different regions.

In [ ]:
fig = px.treemap(vehicles_df_no_outlier, path=['manufacturer', 'model'], values='price',
                  color='price', hover_data=['region'],
                  color_continuous_scale='Magma',
                  color_continuous_midpoint=np.average(vehicles_df_no_outlier['price']), title='Model does vary by price but the high cardinality of it may be bad for a model')
fig.show()
fig.write_image('images/treemap_model_no_outlier.png')

In [ ]:
avg_price_by_manufacturer = vehicles_df_no_outlier.groupby('manufacturer')['price'].mean().sort_values().reset_index()
fig = px.bar(avg_price_by_manufacturer, x='manufacturer', y='price', title='Ford is the most popular yet has a higher average price. Ferrari is missing data')
fig.show()
fig.write_image('images/avg_price_by_manufacturer_no_outlier.png')

In [ ]:
avg_price_by_manufacturer = imputed_vehicle_df.groupby('manufacturer')['price'].mean().sort_values().reset_index()
fig = px.bar(avg_price_by_manufacturer, x='manufacturer', y='price', title='With imputation Telsa takes the highest price')
fig.show()
fig.write_image('images/avg_price_by_manufacturer_imputed.png')

I have seen enough to conclude that our imputed data set is good enough to work with moving forward. I will still build models for each to determine the which is best.

In [ ]:
# Let's do an average price by feature bar char
category_columns_to_ignore = ['VIN', 'region', 'model', 'manufacturer', 'state', 'transmission']
category_columns = list(filter(lambda field: field not in category_columns_to_ignore, list(vehicles_df_no_outlier.select_dtypes(include='object').columns)))
fig = make_subplots(
    rows=len(category_columns), cols=1,
    subplot_titles=("Condition: New has the highest average price, typically better condition equals higher average price",
                    "Cylinders: More cylinders means more power means higher average price",
                    "Drive: 4 wheel drive is more expensive on average",
                    "Fuel: Diesel is the most expensive on average",
                    "Color: Has a negligible affect on average price",
                    "Size: Full size vehicles are more expensive on average",
                    "Title Status: Vehicles with liens are more expensive, likely because the lien needs to be paid off",
                    "Type: Trucks, pickups and offroad vehicles are more expensive on average",
                    )
)

for i, field in enumerate(sorted(category_columns)):
    no_outlier_counts = imputed_vehicle_df[field].value_counts()
    avg_price_by_field = imputed_vehicle_df.groupby(field)['price'].mean().sort_values().reset_index()
    bar_fig = go.Bar(x=avg_price_by_field[field], y=avg_price_by_field['price'])
    fig.add_trace(bar_fig, row=i+1, col=1)

fig.update_layout(height=1400, width=1400, showlegend=False)
fig.show()
fig.write_image("images/average_price_by_field.png")

In [ ]:
# Correlation plots
corr_matrix = imputed_vehicle_df[['price', 'year', 'odometer']].corr(numeric_only=True).round(2)
fig = px.imshow(corr_matrix, title='Newer cars are priced higher, mileage is negatively correlated with year and price',
          color_continuous_scale='RdBu_r', aspect='auto')
fig.show()
fig.write_image('images/correlation.png')

This correlation gave me the idea to add the feature below. Average miles per year.

In [ ]:
current_year = 2025
vehicles_df_no_outlier['age'] = current_year - vehicles_df_no_outlier['year']
imputed_vehicle_df['age'] = current_year - imputed_vehicle_df['year']
vehicles_df_no_outlier['avg_miles_per_year'] = vehicles_df_no_outlier['odometer'] / vehicles_df_no_outlier['age']
imputed_vehicle_df['avg_miles_per_year'] = vehicles_df_no_outlier['odometer'] / vehicles_df_no_outlier['age']

In [ ]:
fig = px.histogram(imputed_vehicle_df, x='avg_miles_per_year', labels={'avg_miles_per_year': 'Avg miles per year', 'count': 'Count'}, title='Average miles per year follows a normal distribution but we have outliers')
fig.show()
fig.write_image('images/avg_miles_per_year_histogram.png')

In [ ]:
fig = px.box(imputed_vehicle_df, x='avg_miles_per_year', labels={'avg_miles_per_year': 'Avg miles per year'}, title='Most vehicles are adding 5k-10k miles a year')
fig.show()
fig.write_image('images/avg_miles_per_year_box.png')

In [ ]:
fig = px.bar(imputed_vehicle_df.groupby('age')['price'].mean().sort_values().reset_index(), y='price', x='age', title='Price decreases as age increases', labels={'price': 'Price', 'age': 'Age'})
fig.show()
fig.write_image('images/average_price_by_age.png')

In [ ]:
# Correlation plots
corr_matrix = imputed_vehicle_df[['price', 'year', 'odometer', 'avg_miles_per_year', 'age']].corr(numeric_only=True).round(2)
fig = px.imshow(corr_matrix, title='Avg miles per year shows no correlation with price. Indicating non-linear relationship',
          color_continuous_scale='RdBu_r', aspect='auto')
fig.show()
fig.write_image('images/correlation_new_features.png')

### Data Preparation

After our initial exploration and fine-tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`.

In the data understanding section, I imputed data, removed outliers and added new features as part. I will repeat the process.

In [ ]:
# Reload dataset
vehicles_df = pd.read_csv('data/vehicles.csv')

In [ ]:
# Drop columns (id, vin, region, model)
vehicles_df.drop(['id', 'VIN', 'region', 'model'], axis=1)

In [ ]:
# Remove outliers
q1 = vehicles_df['price'].quantile(0.25)
q3 = vehicles_df['price'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr)
upper_bound = q3 + (1.5 * iqr)
vehicles_df = vehicles_df.query(f'price >= {lower_bound} and price <= {upper_bound} and year > 1995')

In [204]:
# impute data
# predefine to avoid extra computation
groupby_model_year = vehicles_df.groupby(['model', 'year'])
fields_to_impute = ['condition', 'cylinders', 'drive', 'size', 'type', 'paint_color']

# Impute once since it can be expensive
def load_imputed_data_data_prep():
    for field in fields_to_impute:
        na_filter = vehicles_df[field].isna()
        vehicles_df.loc[na_filter, field] = vehicles_df[na_filter].apply(lambda row: impute_field_by_model_year(row, field), axis=1)

    return vehicles_df

vehicles_df = load_imputed_data_data_prep()

In [205]:
# Check impute change
vehicles_df.isna().mean().round(2)

id              0.00
region          0.00
price           0.00
year            0.00
manufacturer    0.04
model           0.01
condition       0.09
cylinders       0.14
fuel            0.01
odometer        0.01
title_status    0.02
transmission    0.01
VIN             0.38
drive           0.10
size            0.30
type            0.05
paint_color     0.08
state           0.00
dtype: float64

In [ ]:
# Add features
current_year = 2025
vehicles_df['age'] = current_year - vehicles_df['year']
vehicles_df['avg_miles_per_year'] = vehicles_df['odometer'] / vehicles_df['age']

In [206]:
# Drop duplicates and rows with NA
vehicles_df = vehicles_df.drop_duplicates()
vehicles_df = vehicles_df.dropna()

In [207]:
vehicles_df.shape

(152088, 18)

I will let my pipeline do any transformations

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

In [ ]:
# LinearRegression

In [ ]:
# Ridge

In [ ]:
# Lasso

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high-quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight into drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine-tuning their inventory.